### Initial Setup


In [ ]:
%pip install -r requirements.txt

In [ ]:
# load required env vars for ddtrace
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import ddtrace.auto

ddtrace.patch_all()

from ddtrace.llmobs import LLMObs

LLMObs.enable()

### Tracing a Workflow Span

A workflow span reprsents a static sequence. In this example, we are:

1. Taking a query from a user
2. Parsing that quey in a call to openai
3. Sending the parsed query to the Metropolitan Museum of Art API
4. Returning a list of urls to the user.


In [ ]:
import requests
from ddtrace.llmobs.decorators import tool
import json


# API docs: https://metmuseum.github.io/#search
@tool(name="fetch_met_urls")
def fetch_met_urls(query):
    base_url = "https://collectionapi.metmuseum.org/public/collection/v1/search"
    params = {
        "q": query,
        "hasImages": "true",
    }
    response = requests.get(base_url, params=params)
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        objects = response.json()["objectIDs"]
        urls = [
            f"https://www.metmuseum.org/art/collection/search/{objectId}"
            for objectId in objects[:3]
        ]
        LLMObs.annotate(
            input_data=query,
            output_data=urls,
        )
        return urls
    except Exception as e:
        print(f"An error occurred: {e}")
        LLMObs.annotate(status="error", error=e)
        return None

In [ ]:
from openai import OpenAI
import os
import json
from ddtrace.llmobs.decorators import workflow
from ddtrace.llmobs import LLMObs

oai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

sys_prompt = """
	You are a service that provides information about the art in the collection of the Metropolitan Museum of Art.
    You take a free text query and parse it into a format that can be sent to the Met API.
    Parse a query out from the user's message in the following JSON format:

    Prompt: I think I'd like to see some art from the 19th century. Ideally southern European or maybe Northern African.
    Response: {"query": "19th century southern European Northern African"}
	"""


def parse_query(message, prompt=sys_prompt):
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": message},
    ]
    response = (
        oai_client.chat.completions.create(
            messages=messages,
            model="gpt-3.5-turbo",
            response_format={"type": "json_object"},
        )
        .choices[0]
        .message.content
    )
    query = json.loads(response)["query"]
    print(f"Parsed query: {query}")
    return query


@workflow(name="get_art")
def get_art(question, prompt=sys_prompt):
    query = parse_query(question, prompt)
    urls = fetch_met_urls(query)
    LLMObs.annotate(
        input_data=question,
        output_data=json.dumps(urls),
    )
    return urls

In [ ]:
get_art("I think I'd like to see some art from China that features a cat or doc")